In [64]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import permutations

In [65]:
al = set(range(0, 24))
lt = set(range(24, 63))
at = set(range(63, 89))
ca = set(range(89, 127))
ct = set(range(127, 147))

clusters = [lt, at, al, ca, ct]
cluster_str = ['lt', 'at', 'al', 'ca', 'ct']
image_id_to_cluster_id = {}
image_id_to_cluster_str = {}
for cluster_id, cluster in enumerate(clusters):
    for image_id in cluster:
        image_id_to_cluster_id[image_id] = cluster_id
        image_id_to_cluster_str[image_id] = cluster_str[cluster_id]

In [66]:
string_answer_to_code = {
    'All are Same Species': [1, 1, 1],
    'ONLY 1 and 2 are Same Species': [1, 1, 0],
    'ONLY 2 and 3 are Same Species': [0, 1, 1],
    'ONLY 1 and 3 are Same Species': [1, 0, 1],
    'NONE, all are different species': [0, 1, 2]
}

In [67]:
exp_name = 'lt_b-at_a-lt_a'

In [71]:
time_per_question = []
confusion_matrix = defaultdict(lambda: defaultdict(int))

for true_cluster_strs in permutations(exp_name.split("-"), 3):
    with open(f'data/{"-".join(true_cluster_strs)}.json') as file:
        data = json.load(file)
    for worker in data:
        for answer in worker['answer'][1:-1]:

            # time for this question
            time_per_question.append(answer['timeElapsed'])

            # how items are presented in this question
            ordered_image_ids = [item['imageId'] for item in answer['originalOrder']]

            worker_answer = answer['clusters']
            true_cluster_ids = np.array([image_id_to_cluster_id[image_id] for image_id in ordered_image_ids])
            # true_cluster_strs = np.array([image_id_to_cluster_str[image_id] for image_id in ordered_image_ids])
            true_cluster_strs = np.array(true_cluster_strs)

            # sorted_args = np.argsort(true_cluster_ids)
            sorted_args = np.arange(len(true_cluster_ids))
            if exp_name == 'lt-at-lt':
               sorted_args[1], sorted_args[2] = sorted_args[2], sorted_args[1]
            true_cluster_ids = true_cluster_ids[sorted_args]
            true_cluster_strs = true_cluster_strs[sorted_args]
            answer_code = tuple(np.array(string_answer_to_code[worker_answer])[sorted_args])

            if len(np.unique(answer_code)) == 3:
                answer_code = (0, 1, 2)
            true_answer = ''

            if true_cluster_ids[0] == true_cluster_ids[1] == true_cluster_ids[2]:
                true_answer = 'All are Same Species'
            elif true_cluster_ids[0] == true_cluster_ids[1] and true_cluster_ids[1] != true_cluster_ids[2] and true_cluster_ids[0] != true_cluster_ids[2]:
                true_answer = 'ONLY 1 and 2 are Same Species'
            elif true_cluster_ids[1] == true_cluster_ids[2] and true_cluster_ids[1] != true_cluster_ids[0] and true_cluster_ids[0] != true_cluster_ids[2]:
                true_answer = 'ONLY 2 and 3 are Same Species'
            elif true_cluster_ids[0] == true_cluster_ids[2] and true_cluster_ids[1] != true_cluster_ids[0] and true_cluster_ids[1] != true_cluster_ids[2]:
                true_answer = 'ONLY 1 and 3 are Same Species'
            else:
                true_answer = 'NONE, all are different species'

            confusion_matrix[', '.join(true_cluster_strs)][str(answer_code)]  += 1

In [72]:
print(exp_name)
count = pd.DataFrame(confusion_matrix).fillna(0)
pd.DataFrame(confusion_matrix).fillna(0)

lt_b-at_a-lt_a


,"lt_b, at_a, lt_a","lt_b, lt_a, at_a","at_a, lt_b, lt_a","at_a, lt_a, lt_b","lt_a, lt_b, at_a","lt_a, at_a, lt_b"
"(1, 0, 1)",707,34,103,80,140,489
"(1, 1, 1)",64,187,177,123,133,74
"(0, 1, 2)",69,53,40,54,50,67
"(0, 1, 1)",70,71,559,611,79,252
"(1, 1, 0)",90,655,121,132,598,118


In [73]:
emp_obs_prob_mat = pd.DataFrame(confusion_matrix).fillna(0) / pd.DataFrame(confusion_matrix).fillna(0).sum(axis=0)
emp_obs_prob_mat

,"lt_b, at_a, lt_a","lt_b, lt_a, at_a","at_a, lt_b, lt_a","at_a, lt_a, lt_b","lt_a, lt_b, at_a","lt_a, at_a, lt_b"
"(1, 0, 1)",0.707,0.034,0.103,0.080,0.140,0.489
"(1, 1, 1)",0.064,0.187,0.177,0.123,0.133,0.074
"(0, 1, 2)",0.069,0.053,0.040,0.054,0.050,0.067
"(0, 1, 1)",0.070,0.071,0.559,0.611,0.079,0.252
"(1, 1, 0)",0.090,0.655,0.121,0.132,0.598,0.118


In [74]:
print(emp_obs_prob_mat.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.4f}".format,
))  

\begin{tabular}{lrrrrrr}
\toprule
{} &  lt\_b, at\_a, lt\_a &  lt\_b, lt\_a, at\_a &  at\_a, lt\_b, lt\_a &  at\_a, lt\_a, lt\_b &  lt\_a, lt\_b, at\_a &  lt\_a, at\_a, lt\_b \\
\midrule
(1, 0, 1) &            0.7070 &            0.0340 &            0.1030 &            0.0800 &            0.1400 &            0.4890 \\
(1, 1, 1) &            0.0640 &            0.1870 &            0.1770 &            0.1230 &            0.1330 &            0.0740 \\
(0, 1, 2) &            0.0690 &            0.0530 &            0.0400 &            0.0540 &            0.0500 &            0.0670 \\
(0, 1, 1) &            0.0700 &            0.0710 &            0.5590 &            0.6110 &            0.0790 &            0.2520 \\
(1, 1, 0) &            0.0900 &            0.6550 &            0.1210 &            0.1320 &            0.5980 &            0.1180 \\
\bottomrule
\end{tabular}



/var/folders/36/1dhnptp91hzb49wm7_3_1b6m0000gn/T/ipykernel_4991/3078085336.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(emp_obs_prob_mat.to_latex(index=True,


In [75]:
emp_obs_mat = pd.DataFrame(confusion_matrix).fillna(0)
column_sums = emp_obs_mat.sum(axis=0)
column_sums

lt_b, at_a, lt_a    1000
lt_b, lt_a, at_a    1000
at_a, lt_b, lt_a    1000
at_a, lt_a, lt_b    1000
lt_a, lt_b, at_a    1000
lt_a, at_a, lt_b    1000
dtype: int64

In [76]:
emp_var = emp_obs_prob_mat * (1 - emp_obs_prob_mat) * column_sums / (column_sums - 1)
two_sd = 2 * np.sqrt(emp_var / column_sums)

In [52]:
two_sd
print(two_sd.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.4f}".format,
))  

\begin{tabular}{lr}
\toprule
{} &  lt\_b, at\_a, lt\_a \\
\midrule
(0, 1, 1) &            0.0161 \\
(0, 1, 2) &            0.0160 \\
(1, 0, 1) &            0.0288 \\
(1, 1, 0) &            0.0181 \\
(1, 1, 1) &            0.0155 \\
\bottomrule
\end{tabular}



/var/folders/36/1dhnptp91hzb49wm7_3_1b6m0000gn/T/ipykernel_4991/2495288626.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(two_sd.to_latex(index=True,


In [53]:
print(f'Experiment: {exp_name}')

Experiment: lt_b-at_a-lt_a


In [54]:
count.sort_index(ascending=True)

,"lt_b, at_a, lt_a"
"(0, 1, 1)",70
"(0, 1, 2)",69
"(1, 0, 1)",707
"(1, 1, 0)",90
"(1, 1, 1)",64


In [55]:
emp_obs_prob_mat

,"lt_b, at_a, lt_a"
"(0, 1, 1)",0.070
"(0, 1, 2)",0.069
"(1, 0, 1)",0.707
"(1, 1, 0)",0.090
"(1, 1, 1)",0.064


In [56]:
two_sd

,"lt_b, at_a, lt_a"
"(0, 1, 1)",0.016145
"(0, 1, 2)",0.016038
"(1, 0, 1)",0.028800
"(1, 1, 0)",0.018109
"(1, 1, 1)",0.015487
